# Compare bounding box for 2 time steps

In [2]:
import sys
sys.path.append("helpers/")
import helpers.helpers as helpers
import matplotlib
%matplotlib inline
matplotlib.use("nbagg")
from IPython.display import clear_output

from ipyleaflet import (Map, Marker, basemaps, basemap_to_tiles, 
                        TileLayer, ImageOverlay, Polyline, Polygon, 
                        Rectangle,  GeoJSON, WidgetControl,  DrawControl, 
                        LayerGroup, FullScreenControl, interactive)

import ipywidgets as widgets
import datetime

def on_button_clicked(b):

    h = helpers.Helpers(out)
    m1 = h.color_map_nesw_compare(product.value, 
                                  subproduct.value, 
                                  north.value,
                                  east.value, 
                                  south.value, 
                                  west.value,
                                  date1, 
                                  hour1, 
                                  date2, 
                                  hour2)

m = Map(center=(1, -5), zoom=2, layout=dict(width='400px', height='370px'))

dc = DrawControl()
m.add_control(dc)

helpers.Helpers.prepare_map(dc, m)

w = helpers.Widgets()
product = w.product()
subproduct = w.subproduct()
north = w.get_point(38.0, 'North: ')
east = w.get_point(53.0, 'East: ')
south = w.get_point((-36), 'South: ')
west = w.get_point((-19), 'West: ')
date1 = w.get_date(datetime.datetime(2018,1,1), 'Date 1: ')
hour1 = w.get_hour('00', 'Hour 1: ')
date2 = w.get_date(datetime.datetime(2018,1,2), 'Date 2: ')
hour2 = w.get_hour('00', 'Hour 2: ')
button = w.set_up_button(on_button_clicked)
w.display_widget_comparison(product, subproduct, north, east, south, west, 
                            date1, hour1, date2, hour2, button, m)

label = widgets.Label()
helpers.Helpers.mouse_interaction(m, label)

def update_north(x):
    w = helpers.Helpers.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (north, 'value'))

def update_east(x):
    w = helpers.Helpers.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (east, 'value'))

def update_south(x):
    w = helpers.Helpers.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (south, 'value'))

def update_west(x):
    w = helpers.Helpers.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (west, 'value'))    
    

def handle_draw(self, action, geo_json):
    w = helpers.Widgets()
    s = geo_json.get('geometry','type')
    t = s.get('type')
    
    h = helpers.Helpers(out)
    if t == 'Point':
        north, east, south, west = h.get_coords_point(action, geo_json)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)

    if t == 'Polygon':
        north, east, south, west = h.get_coords_polygon(action, geo_json)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)

def update(*args):
    subproduct.options = w.get_subproduct_list(product.value)
    
update()
product.observe(update)

out = w.display_output()

dc.on_draw(handle_draw)


Label(value='')

Output()